### Refrence - https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/4/building-an-agent-reasoning-loop

### Building an Agent Reasoning Loop

In [2]:
import os
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [7]:
#importing get_doc_tools from Utils_FN_QE_tool.py
# from Utils_FN_QE_tool import get_doc_tools


from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)
    
    def vector_query(
        query: str, 
        page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.
    
        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.
    
        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE 
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.
        
        """
    
        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]
        
        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response
        
    
    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )
    
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool

In [9]:
vector_tool, summary_tool = get_doc_tools("./2_ Multidoc_Agentic_RAG/data/metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [10]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

Agent- Query:

In [12]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT are the Architect agent, Engineer agent, QA Engineer, and Project Manager. The Architect agent is responsible for devising technical specifications and system architecture diagrams. The Engineer agent completes development tasks based on provided file structures and function definitions. The QA Engineer generates unit test code and reviews it for bugs. The Project Manager breaks down the project into a task list and assigns tasks to Engineers based on code file analysis.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication between agent roles in MetaGPT"}
=== Function Output ===
The communication between agent roles in MetaGPT is structured and efficient, with eac

In [13]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: 2_ Multidoc_Agentic_RAG/data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-24
last_modified_date: 2024-05-24

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to 

Agent: Chat- if you want to have conversation including memory

In [14]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT are the HumanEval benchmark, the MBPP benchmark, and the self-generated SoftwareDev dataset.
=== LLM Response ===
The evaluation datasets used in MetaGPT include the HumanEval benchmark, the MBPP benchmark, and the self-generated SoftwareDev dataset.


In [15]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over the HumanEval benchmark", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved results of 85.9% and 87.7% over the HumanEval benchmark.
=== LLM Response ===
MetaGPT achieved results of 85.9% and 87.7% over the HumanEval benchmark.


## Lower-Level: Debuggability and Control

In [16]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [17]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [18]:
task
step_output = agent.run_step(task.task_id)

Task(task_id='937947e6-567b-4a47-a1b3-de8971638261', input='Tell me about the agent roles in MetaGPT, and then how they communicate with each other.', memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history'), callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f7775bf7280>, extra_state={'sources': [], 'n_function_calls': 0, 'new_memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')})

In [19]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT are the Architect agent, Engineer agent, and Project Manager. The Architect agent is responsible for devising technical specifications and system architecture diagrams, the Engineer agent completes development tasks, and the Project Manager breaks down the project into tasks for Engineers.


In [20]:
completed_steps = agent.get_completed_steps(task.task_id)


In [23]:
completed_steps

[TaskStepOutput(output=AgentChatResponse(response='assistant: None', sources=[ToolOutput(content='The agent roles in MetaGPT are the Architect agent, Engineer agent, and Project Manager. The Architect agent is responsible for devising technical specifications and system architecture diagrams, the Engineer agent completes development tasks, and the Project Manager breaks down the project into tasks for Engineers.', tool_name='summary_tool_metagpt', raw_input={'input': 'agent roles in MetaGPT'}, raw_output=Response(response='The agent roles in MetaGPT are the Architect agent, Engineer agent, and Project Manager. The Architect agent is responsible for devising technical specifications and system architecture diagrams, the Engineer agent completes development tasks, and the Project Manager breaks down the project into tasks for Engineers.', source_nodes=[NodeWithScore(node=TextNode(id_='5aa102a0-1af2-466d-a131-f3a59e490f86', embedding=None, metadata={'page_label': '1', 'file_name': 'metagp

In [24]:
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")


Num completed for task 937947e6-567b-4a47-a1b3-de8971638261: 1


In [25]:
print(completed_steps[0].output.sources[0].raw_output)

The agent roles in MetaGPT are the Architect agent, Engineer agent, and Project Manager. The Architect agent is responsible for devising technical specifications and system architecture diagrams, the Engineer agent completes development tasks, and the Project Manager breaks down the project into tasks for Engineers.


Upcoming steps

In [26]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 937947e6-567b-4a47-a1b3-de8971638261: 1


TaskStep(task_id='937947e6-567b-4a47-a1b3-de8971638261', step_id='5c464731-d274-45d7-b5d9-5513afa19f1c', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [27]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This shared message pool allows agents to publish structured messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant messages based on their role profiles, enabling them to extract necessary information efficiently. This approach enhances communication efficiency by eliminating the need for one-to-one communication and reducing information overload.


In [28]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This shared message pool allows agents to publish structured messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant messages based on their role profiles, enabling them to extract necessary information efficiently. This approach enhances communication efficiency by eliminating the need for one-to-one communication and reducing information overload.
True


In [29]:
response = agent.finalize_response(task.task_id)

In [30]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This shared message pool allows agents to publish structured messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant messages based on their role profiles, enabling them to extract necessary information efficiently. This approach enhances communication efficiency by eliminating the need for one-to-one communication and reducing information overload.
